In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

In [22]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")


In [23]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}"),
])

In [24]:
output_parser = StrOutputParser()

In [25]:
chain = prompt | model | output_parser

In [26]:
print(chain.invoke(input="What is Langgraph"))

LangChain's LangGraph is a powerful tool for building and managing complex applications leveraging large language models (LLMs).  It's not a single, monolithic component but rather a framework and methodology for structuring LLM-based applications as directed acyclic graphs (DAGs).  This allows for intricate workflows involving multiple LLMs, external data sources, and other processing steps, all orchestrated and managed efficiently.

Here's a breakdown of its key aspects:

**Core Functionality:**

* **DAG Representation:** LangGraph represents applications as DAGs.  Each node in the graph represents a specific processing step, such as calling an LLM, querying a database, or performing some other computation.  The edges define the dependencies between these steps, outlining the flow of data and control. This visual representation makes complex applications much easier to understand, debug, and maintain.

* **Modular Design:**  The modular nature promotes reusability.  Individual nodes 

In [27]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://blog.langchain.dev/langgraph-platform-ga/")
docs = loader.load()

In [28]:
docs

[Document(metadata={'source': 'https://blog.langchain.dev/langgraph-platform-ga/', 'title': 'LangGraph Platform is now Generally Available: Deploy & manage long-running, stateful Agents', 'description': 'LangGraph Platform, our infrastructure for deploying and managing agents at scale, is now generally available. Learn how to deploy', 'language': 'en'}, page_content='\n\n\nLangGraph Platform is now Generally Available: Deploy & manage long-running, stateful Agents\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph Platform is now Generally Available: Deploy & manage long-running, stateful Agents\nLangGraph Platform, our infrastructure for deploying and managing agents at scale, is now generally available. Learn how to deploy\n

In [29]:
# Sayısal temsillere çevirip vektör veri tabanında(faiss) saklayalım
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [30]:
from langchain_community.vectorstores import FAISS


In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [32]:
documents

[Document(metadata={'source': 'https://blog.langchain.dev/langgraph-platform-ga/', 'title': 'LangGraph Platform is now Generally Available: Deploy & manage long-running, stateful Agents', 'description': 'LangGraph Platform, our infrastructure for deploying and managing agents at scale, is now generally available. Learn how to deploy', 'language': 'en'}, page_content='LangGraph Platform is now Generally Available: Deploy & manage long-running, stateful Agents\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph Platform is now Generally Available: Deploy & manage long-running, stateful Agents\nLangGraph Platform, our infrastructure for deploying and managing agents at scale, is now generally available. Learn how to deploy\n\n4 mi

In [33]:
vector = FAISS.from_documents(documents, embeddings)

In [34]:
# Bir retriever chain'i oluşturalım, bu chain gelen soruyu alacak daha sonra ilgili dökümanları bulacak ve bunları orijinal soruyla birlikte LLM'e verecek
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context: 
    <context>
    {context}
    </context>
    Question: {input}
    """
)

In [36]:
# Dokümanları modele geçirmek için zincir kur
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(model, prompt)

In [37]:
from langchain.chains import create_retrieval_chain


In [ ]:
retriever = vector.as_retriever()

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [40]:
response = retrieval_chain.invoke(
    {
        "input":"What is LangGraph"
    }
)
print(response['answer'])

Based on the provided text, LangGraph is a platform for deploying and managing long-running, stateful agents at scale.  It includes a management console to enforce consistency, monitor behavior, and ship updates; a Studio development environment for debugging and iteration; and  supports various deployment options (cloud, hybrid, fully self-hosted).  It's designed to handle the unique challenges of agent infrastructure, such as long-running tasks, asynchronous collaboration, and bursty traffic.  LangGraph Platform is part of a larger suite of LangChain products, but can be used independently.


In [41]:
from langchain_core.documents import Document

my_doc = Document(
    page_content=" LangGraph is a platform for deploying and managing long-running, stateful agents at scale.  It includes a management console to enforce consistency, monitor behavior, and ship updates; a Studio development environment for debugging and iteration; and  supports various deployment options (cloud, hybrid, fully self-hosted)."
)

In [42]:
document_chain.invoke(
    {
        "input":"What is LangGraph?",
        "context":[my_doc]
    }
)

'LangGraph is a platform for deploying and managing long-running, stateful agents at scale.  It includes a management console, a Studio development environment, and supports various deployment options (cloud, hybrid, fully self-hosted).'

Genelde LLM'ler sorulara doğru cevap tam olarak veremediği için bu tekniği kullanıyoruz